<a href="https://colab.research.google.com/github/kundetiA/VoteNet-Point-Cloud-3D-Object-Detection/blob/main/VoteNet_Environment_setup_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Training VoteNet with the SUN RGB-D Dataset**

This Colab notebook demonstrates the training process of VoteNet, a deep learning-based 3D object detection model, using the SUN RGB-D dataset. VoteNet leverages Hough voting to generate 3D object proposals from point clouds, making it effective for indoor scene understanding.

**Can VoteNet Be Used for Outdoor Scenes?**

While VoteNet is primarily optimized for indoor scenes, **it can be adapted for outdoor environments with modifications, such as:**

**Handling Sparse and Large-Scale Point Clouds** – Outdoor LiDAR data (e.g., from self-driving cars) is typically more sparse and unstructured compared to dense indoor point clouds.

**Adjusting Anchor Sizes and Proposal Mechanisms** – Indoor objects are relatively small and well-defined, whereas outdoor objects (e.g., cars, pedestrians) vary significantly in scale.

**Training on Outdoor Dataset**s – VoteNet is trained on datasets like SUN RGB-D, which contain indoor point clouds. To work outdoors, it would need retraining on datasets like KITTI or nuScenes.

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


The VoteNet code is designed to work with Python 3.7, which supports PyTorch 1.13 and CUDA 11.7.

We will install PyTorch 1.13 with CUDA 11.7 inside a virtual environment in Google Colab.

However, since Colab will have CUDA 12.5 pre-installed, there will be a mismatch between the system CUDA and the installed CUDA version.

This discrepancy is likely to cause CUDA compatibility errors during execution.

**Solution**

To avoid CUDA mismatch errors, we need to install a CUDA version in Google Colab that matches the CUDA version used by PyTorch.  

The following commands download and install **CUDA 11.7**, ensuring compatibility with **PyTorch 1.13**, which is built for CUDA 11.7. This helps prevent version mismatch errors in Colab.

In [ ]:
# Installing CUDA 11.7 in google colab
!wget https://developer.download.nvidia.com/compute/cuda/11.7.0/local_installers/cuda_11.7.0_515.43.04_linux.run

--2025-02-15 13:42:38--  https://developer.download.nvidia.com/compute/cuda/11.7.0/local_installers/cuda_11.7.0_515.43.04_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.203.63.26, 125.56.222.3
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.203.63.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3490450898 (3.2G) [application/octet-stream]
Saving to: ‘cuda_11.7.0_515.43.04_linux.run’

cuda_11.7.0_515.43. 100%[===================>]   3.25G  94.3MB/s    in 37s     

2025-02-15 13:43:16 (91.2 MB/s) - ‘cuda_11.7.0_515.43.04_linux.run’ saved [3490450898/3490450898]



In [ ]:
!sudo sh cuda_11.7.0_515.43.04_linux.run --silent --toolkit

Code below updates the environment variables to include the newly installed **CUDA 11.7** toolkit. It modifies the **`PATH`** to include the CUDA binaries and the **`LD_LIBRARY_PATH`** to include the CUDA libraries, ensuring proper access to CUDA tools and libraries.

In [ ]:
import os
os.environ['PATH'] = '/usr/local/cuda-11.7/bin:' + os.environ['PATH']
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.7/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0


CUDA 11.7 is now successfully installed in colab!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Let's install python 3.7 and then create a virtual environment with python 3.7

In [ ]:
# install Python 3.7 with the full installation (including ensurepip)
!sudo apt-get install python3.7-full

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  idle-python3.7 libpython3.7-minimal libpython3.7-stdlib
  libpython3.7-testsuite net-tools python3.7 python3.7-distutils
  python3.7-examples python3.7-gdbm python3.7-lib2to3 python3.7-minimal
  python3.7-tk python3.7-venv
Suggested packages:
  python3.7-dev python3.7-gdbm-dbg binfmt-support tix python3.7-tk-dbg
The following NEW packages will be installed:
  idle-python3.7 libpython3.7-minimal libpython3.7-stdlib
  libpython3.7-testsuite net-tools python3.7 python3.7-distutils
  python3.7-examples python3.7-full python3.7-gdbm python3.7-lib2to3
  python3.7-minimal python3.7-tk python3.7-venv
0 upgraded, 14 newly installed, 0 to remove and 20 not upgraded.
Need to get 11.7 MB of archives.
After this operation, 48.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 net-tools amd64 1.60+git2

We select the installed python version to be applied for this entire notebook session.

**Selection - 3**

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3

There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.11   2         auto mode
  1            /usr/bin/python3.10   1         manual mode
  2            /usr/bin/python3.11   2         manual mode
  3            /usr/bin/python3.7    1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 3
update-alternatives: using /usr/bin/python3.7 to provide /usr/bin/python3 (python3) in manual mode


In [ ]:
!python3.7 -m ensurepip --upgrade
!python3.7 -m pip install --upgrade pip

Looking in links: /tmp/tmpko69q_nt
Processing /tmp/tmpko69q_nt/setuptools-47.1.0-py3-none-any.whl
Processing /tmp/tmpko69q_nt/pip-23.0.1-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [ ]:
!python --version

Python 3.7.17


In [ ]:
!sudo apt-get install python3.7-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.7 libpython3.7-dev
The following NEW packages will be installed:
  libpython3.7 libpython3.7-dev python3.7-dev
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 6,134 kB of archives.
After this operation, 22.9 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.7 amd64 3.7.17-1+jammy1 [1,653 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.7-dev amd64 3.7.17-1+jammy1 [3,981 kB]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-dev amd64 3.7.17-1+jammy1 [500 kB]
Fetched 6,134 kB in 8s (737 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /

In [ ]:
!pip3 install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 37.6 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


The virtual environment is created directly in **Colab** (instead of Google Drive) to accelerate training and model loading, minimizing the slower access times typically associated with Drive storage.

I have also tried creating venv inside drive, but that slowed down the process.

You can experiment with that!

In [ ]:
# Create a virtual environment
!python3.7 -m venv /content/myenv

**Installing necessary dependencies for VoteNet inside the virtual environment.**

In [ ]:
!source /content/myenv/bin/activate && pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 77.8 MB/s eta 0:00:00


In [ ]:
!source /content/myenv/bin/activate && pip install matplotlib opencv-python plyfile 'networkx>=2.2,<2.3' 'trimesh>=2.35.39,<2.35.40'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.0/282.0 kB 34.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 40.7 MB/s eta 0:00:00
  DEPRECATION: networkx is

In [ ]:
!source /content/myenv/bin/activate && pip install torch==1.13.0+cu117 torchvision==0.14.0+cu117 torchaudio==0.13.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 968.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.6/138.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 10.1 MB/s eta 0:00:00


In [ ]:
!source /content/myenv/bin/activate && python -c "import torch; print(f'PyTorch version: {torch.__version__}'); print(f'CUDA version: {torch.version.cuda}')"

PyTorch version: 1.13.0+cu117
CUDA version: 11.7


Now, the **PyTorch CUDA** version and the **Colab system CUDA** are in sync, ensuring compatibility!

Next, we will clone the GitHub **VoteNet** repository to set up the project in the Colab environment.

In [ ]:
!git clone https://github.com/facebookresearch/votenet.git /content/votenet

Cloning into '/content/votenet'...
remote: Enumerating objects: 134, done.
remote: Total 134 (delta 0), reused 0 (delta 0), pack-reused 134 (from 1)
Receiving objects: 100% (134/134), 845.31 KiB | 44.49 MiB/s, done.
Resolving deltas: 100% (30/30), done.


While **VoteNet** relies on PointNet2, **latest version** of **PointNet2** needs to be cloned into the **VoteNet** directory. This ensures we have the most up-to-date version for compatibility.

In [ ]:
!source /content/myenv/bin/activate && git clone https://github.com/erikwijmans/Pointnet2_PyTorch.git /content/votenet/pointnet2/pointnet2_ops/

Cloning into '/content/votenet/pointnet2/pointnet2_ops'...
remote: Enumerating objects: 706, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 706 (delta 0), reused 0 (delta 0), pack-reused 704 (from 2)
Receiving objects: 100% (706/706), 976.27 KiB | 21.69 MiB/s, done.
Resolving deltas: 100% (391/391), done.


In [ ]:
%cd /content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib/

/content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib


In [ ]:
# To check if the Python development headers are installed (Python.h is part of python3.x-dev packages)
!find /usr/include -name "Python.h"

In [ ]:
# Ensures that compilers (like gcc, g++) find Python headers when building C extensions for Python.
# Prevents errors like fatal error: Python.h: No such file or directory when installing packages that require Python C API.

!export CPATH=/usr/include/python3.7m:$CPATH

In [ ]:
# Update package list
!source /content/myenv/bin/activate && sudo apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,315 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,680 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

In [ ]:
# ninja-build is required for compiling PointNet2 efficiently.
!source /content/myenv/bin/activate && sudo apt-get install ninja-build

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  ninja-build
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 111 kB of archives.
After this operation, 358 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ninja-build amd64 1.10.1-1 [111 kB]
Fetched 111 kB in 1s (98.3 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package ninja-build.
(Reading database ... 127737 files and directori

**Installing the requirements for Pointnet2 compilation process.**

It installs all dependencies listed in requirements.txt.
This step does not compile the extension file; it only installs Python dependencies.

In [ ]:
%cd /content/votenet/pointnet2/pointnet2_ops/
!source /content/myenv/bin/activate && pip install -r requirements.txt

Processing ./pointnet2_ops_lib
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.5/289.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.7/299.7 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.1/670.1 kB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 32.8 MB/s eta 

**The following compiles and installs PointNet2 using setup.py. This step is necessary because PointNet2 contains custom CUDA/C++ operations that need to be built before use.**

In [ ]:
!source /content/myenv/bin/activate && python setup.py install

running install
running bdist_egg
running egg_info
creating pointnet2.egg-info
writing pointnet2.egg-info/PKG-INFO
writing dependency_links to pointnet2.egg-info/dependency_links.txt
writing requirements to pointnet2.egg-info/requires.txt
writing top-level names to pointnet2.egg-info/top_level.txt
writing manifest file 'pointnet2.egg-info/SOURCES.txt'
reading manifest file 'pointnet2.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'pointnet2.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/pointnet2
copying pointnet2/train.py -> build/lib/pointnet2
copying pointnet2/__init__.py -> build/lib/pointnet2
copying pointnet2/_version.py -> build/lib/pointnet2
creating build/lib/pointnet2/data
copying pointnet2/data/Indoor3DSemSegLoader.py -> build/lib/pointnet2/data
copying pointnet2/data/data_utils.py -> build/lib/pointnet2/data
copying po

In [ ]:
!source /content/myenv/bin/activate && python -c "import pointnet2_ops; print(pointnet2_ops.__file__)"  # Check the location of the installed module

/content/myenv/lib/python3.7/site-packages/pointnet2_ops/__init__.py


The extension module must be built in place inside the pointnet2_ops package.

This is why we write **sys.path.append(...)** later in file **pointnet2/pointnet2_utils.py** is used to ensure Python can find it.

In [ ]:
# This compiles the CUDA extensions required for PointNet++'s custom layers.

%cd /content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib
!source /content/myenv/bin/activate && python setup.py build_ext --inplace

/content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib
running build_ext
building 'pointnet2_ops._ext' extension
Emitting ninja build file /content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib/build/temp.linux-x86_64-3.7/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
x86_64-linux-gnu-g++ -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 -Wl,-Bsymbolic-functions -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC /content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib/build/temp.linux-x86_64-3.7/pointnet2_ops/_ext-src/src/group_points.o /content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib/build/temp.linux-x86_64-3.7/pointnet2_ops/_ext-src/src/sampling.o /content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib/build/temp.linux-x86_64-3.7/pointnet2_ops/_e

**The Python extension modules were compiled using C++ and CUDA code.
The resulting compiled file is a shared object (.so) file, which can be imported into Python.**

In [ ]:
!source /content/myenv/bin/activate && python -c "import pointnet2_ops; print(pointnet2_ops.__file__)"

/content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib/pointnet2_ops/__init__.py


In [ ]:
!source /content/myenv/bin/activate && ls /content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib/pointnet2_ops

_ext.cpython-37m-x86_64-linux-gnu.so  __init__.py	    pointnet2_utils.py	_version.py
_ext-src			      pointnet2_modules.py  __pycache__


In [ ]:
!ls -l /content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib/pointnet2_ops

total 22304
-rwxr-xr-x 1 root root 22798896 Feb 15 13:29 _ext.cpython-37m-x86_64-linux-gnu.so
drwxr-xr-x 4 root root     4096 Feb 15 13:22 _ext-src
-rw-r--r-- 1 root root      123 Feb 15 13:22 __init__.py
-rw-r--r-- 1 root root     6530 Feb 15 13:22 pointnet2_modules.py
-rw-r--r-- 1 root root    10396 Feb 15 13:22 pointnet2_utils.py
drwxr-xr-x 2 root root     4096 Feb 15 13:29 __pycache__
-rw-r--r-- 1 root root       22 Feb 15 13:22 _version.py


In [ ]:
!ls -lah /content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib/pointnet2_ops

total 22M
drwxr-xr-x 4 root root 4.0K Feb 15 13:29 .
drwxr-xr-x 5 root root 4.0K Feb 15 13:25 ..
-rwxr-xr-x 1 root root  22M Feb 15 13:29 _ext.cpython-37m-x86_64-linux-gnu.so
drwxr-xr-x 4 root root 4.0K Feb 15 13:22 _ext-src
-rw-r--r-- 1 root root  123 Feb 15 13:22 __init__.py
-rw-r--r-- 1 root root 6.4K Feb 15 13:22 pointnet2_modules.py
-rw-r--r-- 1 root root  11K Feb 15 13:22 pointnet2_utils.py
drwxr-xr-x 2 root root 4.0K Feb 15 13:29 __pycache__
-rw-r--r-- 1 root root   22 Feb 15 13:22 _version.py


In [ ]:
# Should print the path to the module # Test if the compiled extension is detected
!source /content/myenv/bin/activate && python -c "import torch; import pointnet2_ops; print(pointnet2_ops.__file__); from pointnet2_ops import pointnet2_utils; print(dir(pointnet2_utils))"

/content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib/pointnet2_ops/__init__.py
['AbstractSet', 'Any', 'AnyStr', 'AsyncContextManager', 'AsyncGenerator', 'AsyncIterable', 'AsyncIterator', 'Awaitable', 'BallQuery', 'ByteString', 'Callable', 'ChainMap', 'ClassVar', 'Collection', 'Container', 'ContextManager', 'Coroutine', 'Counter', 'DefaultDict', 'Deque', 'Dict', 'ForwardRef', 'FrozenSet', 'Function', 'FurthestPointSampling', 'GatherOperation', 'Generator', 'Generic', 'GroupAll', 'GroupingOperation', 'Hashable', 'ItemsView', 'Iterable', 'Iterator', 'KeysView', 'List', 'Mapping', 'MappingView', 'MutableMapping', 'MutableSequence', 'MutableSet', 'NamedTuple', 'NewType', 'NoReturn', 'Optional', 'OrderedDict', 'QueryAndGroup', 'Reversible', 'Sequence', 'Set', 'Sized', 'SupportsAbs', 'SupportsBytes', 'SupportsComplex', 'SupportsFloat', 'SupportsInt', 'SupportsRound', 'TYPE_CHECKING', 'Text', 'ThreeInterpolate', 'ThreeNN', 'Tuple', 'Type', 'TypeVar', 'Union', 'ValuesView', '__builtins__'

Now, to train the VoteNet, to use the pointnet2, we need to add the compiled file path in the file, so that the training starts without any error.

Inside the file votenet/pointnet2/pointnet2_utils.py,

1. Append the path:
**sys.path.append("/content/votenet/pointnet2/pointnet2_ops/pointnet2_ops_lib/pointnet2_ops")**

2. change the code from

**import pointnet2._ext as _ext**

to

**import pointnet2_ops._ext as _ext**

Python will look for _ext.cpython-37m-x86_64-linux-gnu.so inside the pointnet2_ops package.
If it is missing, the import will fail.

In [ ]:
%cd /content/votenet

/content/votenet


Transfer the training dataset (sunrgbd_pc_bbox_votes_50k_v1_train) from Google Drive to the /content/votenet/sunrgbd/ directory in the current Colab environment.

This Notebook does the training with a subset of 500 samples taken from sunrgbd dataset.

You can train with more data.

In [ ]:
cp -r /content/drive/MyDrive/project/sunrgbd_pc_bbox_votes_50k_v1_train /content/votenet/sunrgbd/

In [ ]:
!ls -1 /content/votenet/sunrgbd/sunrgbd_pc_bbox_votes_50k_v1_train | wc -l

1500


Transfer the validation dataset (sunrgbd_pc_bbox_votes_50k_v1_train) from Google Drive to the /content/votenet/sunrgbd/ directory in the current Colab environment.

In [ ]:
cp -r /content/drive/MyDrive/project/sunrgbd_pc_bbox_votes_50k_v1_val  /content/votenet/sunrgbd/

In [ ]:
!ls -1 /content/votenet/sunrgbd/sunrgbd_pc_bbox_votes_50k_v1_val | wc -l

1500


In [ ]:
!source /content/myenv/bin/activate && pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [ ]:
%cd /content/votenet

/content/votenet


### Training begins, the loss values are decreasing!

In [ ]:
!source /content/myenv/bin/activate && CUDA_VISIBLE_DEVICES=0 python train.py --dataset sunrgbd --log_dir log_sunrgbd

2025-02-15 14:51:25.771611: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-15 14:51:25.956770: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 14:51:26.745879: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/lib64-nvidia
2025-02-15 14:51:26.745989: W tensorflow/c

In [ ]:
## dowload the train, validation results, logs (log_sunrgbd) folder

import shutil
from google.colab import files

# Replace 'your_folder' with the actual folder path
folder_path = "/content/votenet/log_sunrgbd"
zip_path = "/content/log_sunrgbd.zip"

# Compress the folder
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)

# Download the zip file
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Visualize the results and also run the inference script with the instructions mentioned in VoteNet Repository.**

In [ ]:
!apt-get install tree -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (54.1 kB/s)
Selecting previously unselected package tree.
(Reading database ... 127749 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!tree /content/votenet/

/content/votenet/
├── CODE_OF_CONDUCT.md
├── CONTRIBUTING.md
├── demo.py
├── doc
│   ├── teaser.jpg
│   └── tips.md
├── eval.py
├── LICENSE
├── models
│   ├── ap_helper.py
│   ├── backbone_module.py
│   ├── boxnet.py
│   ├── dump_helper.py
│   ├── loss_helper_boxnet.py
│   ├── loss_helper.py
│   ├── proposal_module.py
│   ├── __pycache__
│   │   ├── ap_helper.cpython-37.pyc
│   │   ├── backbone_module.cpython-37.pyc
│   │   ├── dump_helper.cpython-37.pyc
│   │   ├── loss_helper.cpython-37.pyc
│   │   ├── proposal_module.cpython-37.pyc
│   │   ├── votenet.cpython-37.pyc
│   │   └── voting_module.cpython-37.pyc
│   ├── votenet.py
│   └── voting_module.py
├── pointnet2
│   ├── _ext_src
│   │   ├── include
│   │   │   ├── ball_query.h
│   │   │   ├── cuda_utils.h
│   │   │   ├── group_points.h
│   │   │   ├── interpolate.h
│   │   │   ├── sampling.h
│   │   │   └── utils.h
│   │   └── src
│   │       ├── ball_query.cpp
│   │       ├── ball_query_gpu.cu
│   │       ├── bindings.cpp
│   │   

**This notebook mainly provides instructions on setting up the environment for VoteNet, resolving PointNet2 compilation issues, and training VoteNet successfully with the SUNRGBD dataset.**

**The Code to run Inference and then to visualize the 3D detections on the Point Cloud will be uploaded soon.**